In [2]:
from langchain.embeddings import GooglePalmEmbeddings
from langchain.llms import GooglePalm

Testing the API

In [48]:
api_key= 'Your Key'
llm = GooglePalm(google_api_key= api_key, temperature = 0.2)
poem =llm('Write a poem on Fanta')
print(poem)

**Fanta**

Fanta, the orange soda,
So sweet and refreshing,
A taste of summer in a can.

Fanta, the drink of choice,
For parties and celebrations,
A way to make any day a special day.

Fanta, the taste of happiness,
A smile on a can,
A reminder that life is good.

Fanta, the drink that brings people together,
A shared experience,
A bond that can't be broken.

Fanta, the orange soda,
A taste of nostalgia,
A reminder of simpler times.

Fanta, the drink that makes you feel good,
A hug in a can,
A friend to the end.


In [4]:
from langchain.utilities import SQLDatabase
from langchain_experimental.sql import SQLDatabaseChain


In [18]:
db_user = "root"
db_password = "dipankar1993"
db_host = "localhost"
db_name = "Aura_Merchandise"

db = SQLDatabase.from_uri(f"mysql+pymysql://{db_user}:{db_password}@{db_host}/{db_name}",sample_rows_in_table_info=5)

print(db.table_info)


CREATE TABLE accessories (
	accessory_id INTEGER NOT NULL AUTO_INCREMENT, 
	type ENUM('Belt','Watch','Sunglasses','Wallet') NOT NULL, 
	brand ENUM('Fossil','Casio','Ray-Ban','Levi') NOT NULL, 
	color ENUM('Red','Blue','Black','White','Brown') NOT NULL, 
	price INTEGER, 
	stock_quantity INTEGER NOT NULL, 
	PRIMARY KEY (accessory_id), 
	CONSTRAINT accessories_chk_1 CHECK ((`price` between 5 and 150))
)ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE utf8mb4_0900_ai_ci

/*
5 rows from accessories table:
accessory_id	type	brand	color	price	stock_quantity
1	Watch	Ray-Ban	Brown	135	49
2	Belt	Fossil	Red	150	38
3	Sunglasses	Levi	Brown	37	37
4	Wallet	Levi	Black	97	31
5	Belt	Casio	Red	112	36
*/


CREATE TABLE discounts (
	discount_id INTEGER NOT NULL AUTO_INCREMENT, 
	merchandise_id INTEGER NOT NULL, 
	merchandise_type ENUM('T-Shirt','Hat','Shoe','Accessory') NOT NULL, 
	pct_discount DECIMAL(5, 2), 
	PRIMARY KEY (discount_id), 
	CONSTRAINT discounts_chk_1 CHECK ((`pct_discount` between 0 and 100))

In [6]:
from langchain_experimental.sql import SQLDatabaseChain
db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True)

qns1 = db_chain("How many t-shirts do we have left for nike in extra small size and white color?")



> Entering new SQLDatabaseChain chain...
How many t-shirts do we have left for nike in extra small size and white color?
SQLQuery:SELECT stock_quantity FROM t_shirts WHERE brand = 'Nike' AND color = 'White' AND size = 'XS'
SQLResult: [(58,)]
Answer:58
> Finished chain.


In [7]:
q1 = db_chain("What is the  stock quantity of all the Levi shirts ")



> Entering new SQLDatabaseChain chain...
What is the  stock quantity of all the Levi shirts 
SQLQuery:SELECT SUM(stock_quantity) FROM t_shirts WHERE brand = 'Levi'
SQLResult: [(Decimal('682'),)]
Answer:682
> Finished chain.


In [23]:
t1 = db_chain("What is the  total stock quantity of all the Levi products accross the store")



> Entering new SQLDatabaseChain chain...
What is the  total stock quantity of all the Levi products accross the store
SQLQuery:SELECT SUM(stock_quantity) FROM t_shirts WHERE brand = 'Levi'
SQLResult: [(Decimal('682'),)]
Answer:682
> Finished chain.


In [24]:
t1 = db_chain("What is the  quantity of reebok blue shoes")



> Entering new SQLDatabaseChain chain...
What is the  quantity of reebok blue shoes
SQLQuery:SELECT stock_quantity FROM shoes WHERE brand = 'Reebok' AND color = 'Blue'
SQLResult: [(37,), (48,)]
Answer:37
> Finished chain.


In [31]:
t2 = db_chain("How many Adidas products are there in total across all categories?")



> Entering new SQLDatabaseChain chain...
How many Adidas products are there in total across all categories?
SQLQuery:SELECT count(*) FROM (
	SELECT * FROM accessories WHERE brand = 'Adidas'
	UNION ALL
	SELECT * FROM hats WHERE brand = 'Adidas'
	UNION ALL
	SELECT * FROM shoes WHERE brand = 'Adidas'
	UNION ALL
	SELECT * FROM t_shirts WHERE brand = 'Adidas'
) AS t;
SQLResult: [(33,)]
Answer:33
> Finished chain.


In [32]:
t3 = db_chain("What is the total value of black shoes in our inventory?")



> Entering new SQLDatabaseChain chain...
What is the total value of black shoes in our inventory?
SQLQuery:SELECT SUM(price) FROM shoes WHERE color = 'Black'
SQLResult: [(Decimal('626'),)]
Answer:626
> Finished chain.


In [33]:
t4= db_chain("If we apply discounts, how much revenue will we generate from all Puma hats?")



> Entering new SQLDatabaseChain chain...
If we apply discounts, how much revenue will we generate from all Puma hats?
SQLQuery:SELECT SUM(price * (1 - pct_discount)) FROM hats JOIN discounts ON hats.hat_id = discounts.merchandise_id WHERE hats.brand = 'Puma' AND discounts.merchandise_type = 'Hat' AND CURDATE() BETWEEN discounts.start_date AND discounts.end_date

OperationalError: (pymysql.err.OperationalError) (1054, "Unknown column 'discounts.start_date' in 'where clause'")
[SQL: SELECT SUM(price * (1 - pct_discount)) FROM hats JOIN discounts ON hats.hat_id = discounts.merchandise_id WHERE hats.brand = 'Puma' AND discounts.merchandise_type = 'Hat' AND CURDATE() BETWEEN discounts.start_date AND discounts.end_date]
(Background on this error at: https://sqlalche.me/e/20/e3q8)

The langugae model could not provide a correct SQL code for the query. So we will have to create a custom query and feed it to the model.

In [34]:
t5 = db_chain('How many Ray-Ban sunglasses do we have in stock')



> Entering new SQLDatabaseChain chain...
How many Ray-Ban sunglasses do we have in stock
SQLQuery:SELECT stock_quantity FROM accessories WHERE brand = 'Ray-Ban' AND type = 'Sunglasses'
SQLResult: [(41,)]
Answer:41
> Finished chain.


In [36]:
t6= db_chain('What is the total price of size 10 Nike shoes?')



> Entering new SQLDatabaseChain chain...
What is the total price of size 10 Nike shoes?
SQLQuery:SELECT SUM(price) FROM shoes WHERE size = 10 AND brand = 'Nike'
SQLResult: [(Decimal('92'),)]
Answer:92
> Finished chain.


In [38]:
t7= db_chain('If we sell all Levi products at the current price, what would be the total revenue?')



> Entering new SQLDatabaseChain chain...
If we sell all Levi products at the current price, what would be the total revenue?
SQLQuery:SELECT SUM(price) FROM t_shirts WHERE brand = 'Levi'
SQLResult: [(Decimal('381'),)]
Answer:381
> Finished chain.


In [39]:
t8= db_chain('What is the total revenue potential for all shoes, assuming they are sold at a discount?')



> Entering new SQLDatabaseChain chain...
What is the total revenue potential for all shoes, assuming they are sold at a discount?
SQLQuery:SELECT SUM(price * (1 - pct_discount)) FROM shoes JOIN discounts ON shoes.shoe_id = discounts.merchandise_id WHERE merchandise_type = 'Shoe';
SQLResult: [(Decimal('-380.00'),)]
Answer:-380.00
> Finished chain.


In [40]:
t9 = db_chain('How many unique colors of accessories are available for each type?')



> Entering new SQLDatabaseChain chain...
How many unique colors of accessories are available for each type?
SQLQuery:SELECT type, count(distinct color) AS num_colors FROM accessories GROUP BY type
SQLResult: [('Belt', 5), ('Watch', 5), ('Sunglasses', 4), ('Wallet', 4)]
Answer:Belt: 5, Watch: 5, Sunglasses: 4, Wallet: 4
> Finished chain.


In [41]:
t10 = db_chain('Find the average price of all items in each category (t-shirts, hats, shoes, accessories) that have at least 50 items in stock.')



> Entering new SQLDatabaseChain chain...
Find the average price of all items in each category (t-shirts, hats, shoes, accessories) that have at least 50 items in stock.
SQLQuery:SELECT AVG(price) AS average_price, merchandise_type
FROM (
	SELECT t_shirt_id, price, stock_quantity
	FROM t_shirts
	WHERE stock_quantity >= 50
	UNION ALL
	SELECT hat_id, price, stock_quantity
	FROM hats
	WHERE stock_quantity >= 50
	UNION ALL
	SELECT shoe_id, price, stock_quantity
	FROM shoes
	WHERE stock_quantity >= 50
	UNION ALL
	SELECT accessory_id, price, stock_quantity
	FROM accessories
	WHERE stock_quantity >= 50
) AS t
GROUP BY merchandise_type

OperationalError: (pymysql.err.OperationalError) (1054, "Unknown column 'merchandise_type' in 'field list'")
[SQL: SELECT AVG(price) AS average_price, merchandise_type
FROM (
	SELECT t_shirt_id, price, stock_quantity
	FROM t_shirts
	WHERE stock_quantity >= 50
	UNION ALL
	SELECT hat_id, price, stock_quantity
	FROM hats
	WHERE stock_quantity >= 50
	UNION ALL
	SELECT shoe_id, price, stock_quantity
	FROM shoes
	WHERE stock_quantity >= 50
	UNION ALL
	SELECT accessory_id, price, stock_quantity
	FROM accessories
	WHERE stock_quantity >= 50
) AS t
GROUP BY merchandise_type]
(Background on this error at: https://sqlalche.me/e/20/e3q8)

Again we can see that the model fails to provide the desired output. So we will provide the custom query to the model so that it can train itself better. Let us keep asking it some hard questions.

In [42]:
t11 = db_chain('Which brand has the highest total stock quantity across all categories?')



> Entering new SQLDatabaseChain chain...
Which brand has the highest total stock quantity across all categories?
SQLQuery:SELECT brand, SUM(stock_quantity) AS total_stock_quantity
FROM accessories
FULL OUTER JOIN hats ON accessories.brand = hats.brand
FULL OUTER JOIN shoes ON accessories.brand = shoes.brand
FULL OUTER JOIN t_shirts ON accessories.brand = t_shirts.brand
GROUP BY brand
ORDER BY total_stock_quantity DESC
LIMIT 1

ProgrammingError: (pymysql.err.ProgrammingError) (1064, "You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near 'OUTER JOIN hats ON accessories.brand = hats.brand\nFULL OUTER JOIN shoes ON acces' at line 3")
[SQL: SELECT brand, SUM(stock_quantity) AS total_stock_quantity
FROM accessories
FULL OUTER JOIN hats ON accessories.brand = hats.brand
FULL OUTER JOIN shoes ON accessories.brand = shoes.brand
FULL OUTER JOIN t_shirts ON accessories.brand = t_shirts.brand
GROUP BY brand
ORDER BY total_stock_quantity DESC
LIMIT 1]
(Background on this error at: https://sqlalche.me/e/20/f405)

In [44]:
t12= db_chain('How many different brands of Medium sized hats do we have?')



> Entering new SQLDatabaseChain chain...
How many different brands of Medium sized hats do we have?
SQLQuery:SELECT count(distinct brand) FROM hats WHERE size = 'Medium'
SQLResult: [(4,)]
Answer:4
> Finished chain.


In [45]:
t13 = db_chain('What is the total number of items in stock for each color across all merchandise categories?')



> Entering new SQLDatabaseChain chain...
What is the total number of items in stock for each color across all merchandise categories?
SQLQuery:SELECT color, SUM(stock_quantity) AS total_quantity
FROM accessories
FULL OUTER JOIN hats ON accessories.accessory_id = hats.hat_id
FULL OUTER JOIN shoes ON accessories.accessory_id = shoes.shoe_id
FULL OUTER JOIN t_shirts ON accessories.accessory_id = t_shirts.t_shirt_id
GROUP BY color
ORDER BY total_quantity DESC

ProgrammingError: (pymysql.err.ProgrammingError) (1064, "You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near 'OUTER JOIN hats ON accessories.accessory_id = hats.hat_id\nFULL OUTER JOIN shoes ' at line 3")
[SQL: SELECT color, SUM(stock_quantity) AS total_quantity
FROM accessories
FULL OUTER JOIN hats ON accessories.accessory_id = hats.hat_id
FULL OUTER JOIN shoes ON accessories.accessory_id = shoes.shoe_id
FULL OUTER JOIN t_shirts ON accessories.accessory_id = t_shirts.t_shirt_id
GROUP BY color
ORDER BY total_quantity DESC]
(Background on this error at: https://sqlalche.me/e/20/f405)

In [8]:
q2 = db_chain("What is the total stock quantity of all the Levi shirts ")



> Entering new SQLDatabaseChain chain...
What is the total stock quantity of all the Levi shirts 
SQLQuery:SELECT SUM(stock_quantity) FROM t_shirts WHERE brand = 'Levi'
SQLResult: [(Decimal('682'),)]
Answer:682
> Finished chain.


In [9]:
q2 = db_chain("List down the total number of unique brands present in stock ")



> Entering new SQLDatabaseChain chain...
List down the total number of unique brands present in stock 
SQLQuery:SELECT count(DISTINCT brand) FROM accessories
SQLResult: [(4,)]
Answer:4
> Finished chain.


Now i will take all the questions the model was not able to answer, I will write a custom query for the same and make sure that they are running on my SQL work bench. Then i will provide them to the model for better training. 

Now if i ask similar complicated questions to my model it will be able to better answer them.

In [50]:
few_shots = [
    {
        'Question': "How many Adidas products are there in total across all categories?",
        'SQLQuery': "SELECT SUM(stock_quantity) FROM (SELECT stock_quantity FROM t_shirts WHERE brand = 'Adidas' UNION ALL SELECT stock_quantity FROM hats WHERE brand = 'Adidas' UNION ALL SELECT stock_quantity FROM shoes WHERE brand = 'Adidas' UNION ALL SELECT stock_quantity FROM accessories WHERE brand = 'Adidas') as total_adidas",
        'SQLResult': "Result of the SQL query",
        'Answer': "356"
    },
    {
        'Question': "What is the total value of black shoes in our inventory?",
        'SQLQuery': "SELECT SUM(price * stock_quantity) FROM shoes WHERE color = 'Black'",
        'SQLResult': "Result of the SQL query",
        'Answer': "15230"
    },
    {
        'Question': "If we apply discounts, how much revenue will we generate from all Puma hats?",
        'SQLQuery': """SELECT SUM(total_revenue) AS total_revenue_generated
                        FROM (
                            SELECT 
                                hats.hat_id, 
                                hats.price * hats.stock_quantity * COALESCE((100 - discounts.pct_discount)/100, 1) AS total_revenue
                            FROM hats
                            LEFT JOIN discounts 
                                ON hats.hat_id = discounts.merchandise_id 
                                AND discounts.merchandise_type = 'Hat'
                            WHERE hats.brand = 'Puma'
                        ) AS revenue_table;""",
        'SQLResult': "Result of the SQL query",
        'Answer': "4993.00"
    },
    {
        'Question': "How many Ray-Ban sunglasses do we have in stock?",
        'SQLQuery': "SELECT SUM(stock_quantity) FROM accessories WHERE type = 'Sunglasses' AND brand = 'Ray-Ban'",
        'SQLResult': "Result of the SQL query",
        'Answer': "41"
    },
    {
        'Question': "What is the total price of size 10 Nike shoes?",
        'SQLQuery': "SELECT SUM(stock_quantity * price) FROM shoes WHERE size = 10 AND brand = 'Nike'",
        'SQLResult': "Result of the SQL query",
        'Answer': "2420"
    },
    {
        'Question': "If we sell all Levi products at the current price, what would be the total revenue?",
        'SQLQuery': """
                SELECT SUM(total_revenue) AS total_revenue_generated
                        FROM (
                            SELECT SUM(price * stock_quantity) AS total_revenue
                            FROM t_shirts
                            WHERE brand = 'Levi'
                            UNION ALL
                            SELECT SUM(price * stock_quantity)
                            FROM hats
                            WHERE brand = 'Levi'
                            UNION ALL
                            SELECT SUM(price * stock_quantity)
                            FROM shoes
                            WHERE brand = 'Levi'
                            UNION ALL
                            SELECT SUM(price * stock_quantity)
                            FROM accessories
                            WHERE brand = 'Levi'
                        ) AS revenue_table;
                    """,
        'SQLResult': "The total revenue generated from selling all Levi products at the current price.",
        'Answer': "58388"
    },

    {
        'Question': "What is the total revenue potential for all shoes, assuming they are sold at a discount?",
        'SQLQuery': """SELECT SUM(a.total_price * ((100 - COALESCE(d.pct_discount, 0)) / 100)) as total_revenue
                       FROM (SELECT shoe_id, SUM(price * stock_quantity) as total_price FROM shoes GROUP BY shoe_id) a
                       LEFT JOIN discounts d ON a.shoe_id = d.merchandise_id AND d.merchandise_type = 'Shoe'""",
        'SQLResult': "Result of the SQL query",
        'Answer': "60605.0"
    },
    {
        'Question': "Name all the unique brands in each category of our inventory.",
        'SQLQuery': """SELECT 'T-Shirt' AS Category, brand FROM t_shirts GROUP BY brand
                       UNION
                       SELECT 'Hat' AS Category, brand FROM hats GROUP BY brand
                       UNION
                       SELECT 'Shoe' AS Category, brand FROM shoes GROUP BY brand
                       UNION
                       SELECT 'Accessory' AS Category, brand FROM accessories GROUP BY brand""",
        'SQLResult': "'T-Shirt', 'Van Huesen'\n'T-Shirt', 'Levi'\n'T-Shirt', 'Nike'\n'T-Shirt', 'Adidas'\n'Hat', 'New Era'\n'Hat', 'Nike'\n'Hat', 'Adidas'\n'Hat', 'Puma'\n'Shoe', 'Nike'\n'Shoe', 'Adidas'\n'Shoe', 'Puma'\n'Shoe', 'Reebok'\n'Accessory', 'Ray-Ban'\n'Accessory', 'Fossil'\n'Accessory', 'Levi'\n'Accessory', 'Casio'",
        'Answer': "T-Shirts: Van Huesen, Levi, Nike, Adidas; Hats: New Era, Nike, Adidas, Puma; Shoes: Nike, Adidas, Puma, Reebok; Accessories: Ray-Ban, Fossil, Levi, Casio"
    },
    {
        'Question': "Find the average price of all items in each category (t-shirts, hats, shoes, accessories) that have at least 50 items in stock.",
        'SQLQuery': """SELECT 'T-Shirts' as category, AVG(price) as avg_price FROM t_shirts WHERE stock_quantity >= 50
                       UNION ALL
                       SELECT 'Hats', AVG(price) FROM hats WHERE stock_quantity >= 50
                       UNION ALL
                       SELECT 'Shoes', AVG(price) FROM shoes WHERE stock_quantity >= 50
                       UNION ALL
                       SELECT 'Accessories', AVG(price) FROM accessories WHERE stock_quantity >= 50""",
        'SQLResult': "Result of the SQL query",
        'Answer': "Average prices: T-Shirts - $30.45, Hats - $7, Shoes - $82, Accessories - $11"
    },
    {
        'Question': "Which brand has the highest total stock quantity across all categories?",
        'SQLQuery': """SELECT brand, SUM(stock) as total_stock FROM
                       (SELECT brand, SUM(stock_quantity) as stock FROM t_shirts GROUP BY brand
                        UNION ALL
                        SELECT brand, SUM(stock_quantity) FROM hats GROUP BY brand
                        UNION ALL
                        SELECT brand, SUM(stock_quantity) FROM shoes GROUP BY brand
                        UNION ALL
                        SELECT brand, SUM(stock_quantity) FROM accessories GROUP BY brand) as all_items
                       GROUP BY brand ORDER BY total_stock DESC LIMIT 1""",
        'SQLResult': "Result of the SQL query",
        'Answer': "Nike with 1518 items"
    },
    {
        'Question': "How many different brands of 'Medium' sized hats do we have?",
        'SQLQuery': "SELECT COUNT(DISTINCT brand) FROM hats WHERE size = 'Medium'",
        'SQLResult': "Result of the SQL query",
        'Answer': "4"
    },
    {
        'Question': "What is the total number of items in stock for each color across all merchandise categories?",
        'SQLQuery': """SELECT color, SUM(stock) as total_stock FROM
                       (SELECT color, SUM(stock_quantity) as stock FROM t_shirts GROUP BY color
                        UNION ALL
                        SELECT color, SUM(stock_quantity) FROM hats GROUP BY color
                        UNION ALL
                        SELECT color, SUM(stock_quantity) FROM shoes GROUP BY color
                        UNION ALL
                        SELECT color, SUM(stock_quantity) FROM accessories GROUP BY color) as all_colors
                       GROUP BY color""",
        'SQLResult': "Result of the SQL query",
        'Answer': "Black: 1252, Blue: 1555, Red: 1693, White: 1733, Brown=439"
    },
   
    {
        'Question': "What are the top 5 items from each category based on stock quantity?",
        'SQLQuery': """SELECT category, brand, stock_quantity, rank_item
                       FROM (
                           SELECT category, brand, stock_quantity,
                                  RANK() OVER (PARTITION BY category ORDER BY stock_quantity DESC) as rank_item
                           FROM (
                               SELECT 'T-Shirt' as category, brand, stock_quantity FROM t_shirts
                               UNION ALL
                               SELECT 'Hat' as category, brand, stock_quantity FROM hats
                               UNION ALL
                               SELECT 'Shoe' as category, brand, stock_quantity FROM shoes
                               UNION ALL
                               SELECT 'Accessory' as category, brand, stock_quantity FROM accessories
                           ) as all_products
                       ) as ranked_products
                       WHERE rank_item <= 5;""",
        'SQLResult': "'Accessory', 'Fossil', '50', '1'\n'Accessory', 'Ray-Ban', '49', '2'\n'Accessory', 'Fossil', '48', '3'\n'Accessory', 'Levi', '46', '4'\n'Accessory', 'Levi', '45', '5'\n'Hat', 'New Era', '50', '1'\n'Hat', 'Adidas', '49', '2'\n'Hat', 'Puma', '49', '2'\n'Hat', 'Puma', '49', '2'\n'Hat', 'Nike', '46', '5'\n'Shoe', 'Adidas', '50', '1'\n'Shoe', 'Reebok', '48', '2'\n'Shoe', 'Puma', '47', '3'\n'Shoe', 'Puma', '47', '3'\n'Shoe', 'Nike', '45', '5'\n'T-Shirt', 'Nike', '100', '1'\n'T-Shirt', 'Van Huesen', '100', '1'\n'T-Shirt', 'Levi', '99', '3'\n'T-Shirt', 'Adidas', '98', '4'\n'T-Shirt', 'Adidas', '94', '5'",
        'Answer': "The top 5 items in each category, ranked by stock quantity, include: Accessories by Fossil and Ray-Ban; Hats by New Era, Adidas, and Puma; Shoes by Adidas, Reebok, and Puma; T-Shirts by Nike, Van Huesen, Levi, and Adidas."
    }
    
]


In [51]:
from langchain.prompts import SemanticSimilarityExampleSelector
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from chromadb.utils import embedding_functions

I will be using a Sentence transformer to embed the queries. What this will do is that it will turn my query into vector form and it can understand similar sentences based on the vectors using cosine similarity. 

Then i will create a vector database using ChromaDB and store it.
The vector database will look at the query and will give you a similar looking query from its database, in this case few_shot examples which i created.
to_vectorize will create a string of the query and keep it in the database.

In [52]:
from langchain.embeddings import SentenceTransformerEmbeddings

embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
to_vectorize = [" ".join(example.values()) for example in few_shots]
to_vectorize

["How many Adidas products are there in total across all categories? SELECT SUM(stock_quantity) FROM (SELECT stock_quantity FROM t_shirts WHERE brand = 'Adidas' UNION ALL SELECT stock_quantity FROM hats WHERE brand = 'Adidas' UNION ALL SELECT stock_quantity FROM shoes WHERE brand = 'Adidas' UNION ALL SELECT stock_quantity FROM accessories WHERE brand = 'Adidas') as total_adidas Result of the SQL query 356",
 "What is the total value of black shoes in our inventory? SELECT SUM(price * stock_quantity) FROM shoes WHERE color = 'Black' Result of the SQL query 15230",
 "If we apply discounts, how much revenue will we generate from all Puma hats? SELECT SUM(total_revenue) AS total_revenue_generated\n                        FROM (\n                            SELECT \n                                hats.hat_id, \n                                hats.price * hats.stock_quantity * COALESCE((100 - discounts.pct_discount)/100, 1) AS total_revenue\n                            FROM hats\n         

In [53]:

vectorstore = Chroma.from_texts(to_vectorize, embeddings, metadatas=few_shots)

The Semantic example selector can pull out similar looking queries from my fewshots examples based on the question i am askig it. The examples it pulls out is may not be exactly same but will give you similar examples from few shots.

In [54]:
example_selector = SemanticSimilarityExampleSelector(
    vectorstore=vectorstore,
    k=2,
)

example_selector.select_examples({"Question": "How many Adidas T shirts I have left in my store?"})

[{'Answer': 'T-Shirts: Van Huesen, Levi, Nike, Adidas; Hats: New Era, Nike, Adidas, Puma; Shoes: Nike, Adidas, Puma, Reebok; Accessories: Ray-Ban, Fossil, Levi, Casio',
  'Question': 'Name all the unique brands in each category of our inventory.',
  'SQLQuery': "SELECT 'T-Shirt' AS Category, brand FROM t_shirts GROUP BY brand\n                       UNION\n                       SELECT 'Hat' AS Category, brand FROM hats GROUP BY brand\n                       UNION\n                       SELECT 'Shoe' AS Category, brand FROM shoes GROUP BY brand\n                       UNION\n                       SELECT 'Accessory' AS Category, brand FROM accessories GROUP BY brand",
  'SQLResult': "'T-Shirt', 'Van Huesen'\n'T-Shirt', 'Levi'\n'T-Shirt', 'Nike'\n'T-Shirt', 'Adidas'\n'Hat', 'New Era'\n'Hat', 'Nike'\n'Hat', 'Adidas'\n'Hat', 'Puma'\n'Shoe', 'Nike'\n'Shoe', 'Adidas'\n'Shoe', 'Puma'\n'Shoe', 'Reebok'\n'Accessory', 'Ray-Ban'\n'Accessory', 'Fossil'\n'Accessory', 'Levi'\n'Accessory', 'Casio'"

In [56]:
from langchain.prompts import FewShotPromptTemplate
from langchain.chains.sql_database.prompt import PROMPT_SUFFIX, _mysql_prompt

There are various custom templetes available on langchain. Please take a look how you can use them.

In [ ]:
print(_mysql_prompt)

You are a MySQL expert. Given an input question, first create a syntactically correct MySQL query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most {top_k} results using the LIMIT clause as per MySQL. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in backticks (`) to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use CURDATE() function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: Result of

In [ ]:
from langchain.prompts.prompt import PromptTemplate

example_prompt = PromptTemplate(
    input_variables=["Question", "SQLQuery", "SQLResult","Answer",],
    template="\nQuestion: {Question}\nSQLQuery: {SQLQuery}\nSQLResult: {SQLResult}\nAnswer: {Answer}",
)

In [ ]:
few_shot_prompt= FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix=_mysql_prompt,
    suffix=PROMPT_SUFFIX,
    input_variables=["input","table_info","top_k"]
)

After I am satisfied with my prompt template I have created a new chain which will ping the LLM model with the custom prompt that i have created and will take the response from there and will run it on my SQL server.

In [ ]:
new_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True, prompt=few_shot_prompt)


In [ ]:
new_chain("How many white color Levi's shirt I have?")



> Entering new SQLDatabaseChain chain...
How many white color Levi's shirt I have?
SQLQuery:SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'
SQLResult: [(Decimal('148'),)]
Answer:148
> Finished chain.


{'query': "How many white color Levi's shirt I have?", 'result': '148'}

In [ ]:
new_chain("How much is the price of the inventory for all small size t-shirts?")




> Entering new SQLDatabaseChain chain...
How much is the price of the inventory for all small size t-shirts?
SQLQuery:SELECT sum(price*stock_quantity) as total_price from t_shirts where size = 'S'
SQLResult: [(Decimal('9959'),)]
Answer:9959
> Finished chain.


{'query': 'How much is the price of the inventory for all small size t-shirts?',
 'result': '9959'}

In [ ]:
new_chain.run('How much revenue  our store will generate by selling all Van Heuson TShirts without discount?')



> Entering new SQLDatabaseChain chain...
How much revenue  our store will generate by selling all Van Heuson TShirts without discount?
SQLQuery:SELECT sum(price*stock_quantity) as total_revenue from t_shirts where brand = 'Van Huesen'
SQLResult: [(Decimal('32890'),)]
Answer:32890
> Finished chain.


'32890'